<a href="https://colab.research.google.com/github/cisirtraining/summerschool/blob/master/Day1_Introduction_to_DL_framework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Object detection**

In [15]:
#author abukhari

import numpy as np
import cv2

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

from os import listdir
#-----------------------

color = (0, 255, 0) # BGR
# (255,0,0) Blue
# (0, 255, 0) Green

face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/detection_project/haarcascade_frontalface_default.xml')

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    #preprocess_input normalizes input in scale of [-1, +1]. You must apply same normalization in prediction.
    img = preprocess_input(img)
    return img

def loadVggFaceModel(): # class: Encapsulates the Mask myDeepFace model functionality.
	model = Sequential()
	model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(Convolution2D(4096, (7, 7), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(4096, (1, 1), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(2622, (1, 1)))
	model.add(Flatten())
	model.add(Activation('softmax'))
	
	#you can download pretrained weights from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
	from tensorflow.keras.models import model_from_json
	model.load_weights('/content/drive/My Drive/detection_project/vgg_face_weights.h5')
	
	vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
	
	return vgg_face_descriptor

model = loadVggFaceModel()

#put your employee pictures in this path as name_of_employee.jpg
employee_pictures = "/content/drive/My Drive/detection_project/database"

employees = dict()

for file in listdir(employee_pictures):
	employee, extension = file.split(".")
	employees[employee] = model.predict(preprocess_image(employee_pictures+'/%s.jpg' % (employee)))[0,:]
	
print("employee representations retrieved successfully")

def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))


employee representations retrieved successfully


In [ ]:
# cap = cv2.VideoCapture(0) #webcam
# #cap = cv2.VideoCapture('C:/Users/IS96273/Desktop/zuckerberg.mp4') #video

# while(True):
# 	ret, img = cap.read() # XXX: what 'img' format
# 	#img = cv2.resize(img, (640, 360))
# 	faces = face_cascade.detectMultiScale(img, 1.3, 5)
	
# 	for (x,y,w,h) in faces:
# 		if w > 130: 
# 			#cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image
			
# 			detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
# 			detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224
			
# 			img_pixels = image.img_to_array(detected_face)
# 			img_pixels = np.expand_dims(img_pixels, axis = 0)
# 			#img_pixels /= 255
# 			#employee dictionary is using preprocess_image and it normalizes in scale of [-1, +1]
# 			img_pixels /= 127.5
# 			img_pixels -= 1
			
# 			captured_representation = model.predict(img_pixels)[0,:]
			
# 			found = 0
# 			for i in employees:
# 				employee_name = i
# 				representation = employees[i]
				
# 				similarity = findCosineSimilarity(representation, captured_representation)
# 				if(similarity < 0.30):
# 					cv2.putText(img, employee_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
					
# 					found = 1
# 					break
					
# 			#connect face and text
# 			cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
# 			cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
		
# 			if(found == 0): #if found image is not in employee database
# 				cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
	
# 	cv2.imshow('img',img)
	
# 	if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
# 		break
	
# #kill open cv things		
# cap.release()
# cv2.destroyAllWindows()

**Object detection on colab**

In [19]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        // create html elements
        const div = document.createElement('div');
        const video = document.createElement('video');
        video.style.display = 'block';

        // request the stream. This will ask for Permission to access 
        // a connected Camera/Webcam
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        // show the HTML elements
        document.body.appendChild(div);
        div.appendChild(video);
        // display the stream
        video.srcObject = stream;
        await video.play();

        // Resize the output (of Colab Notebook Cell) to fit the video element.
        // google.colab.output
        //     .setIfrmeHeight(document.documentElement.scrollHeight, true);

        // capture 5 frames (for test)
        for (let i = 0; i < 5; i++) {
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            img = canvas.toDataURL('image/jpeg', quality);

            // Call a python function and send this image
            google.colab.kernel.invokeFunction('notebook.run_algo', [img], {});

            // wait for X miliseconds second, before next capture
            await new Promise(resolve => setTimeout(resolve, 250));
        }

        stream.getVideoTracks()[0].stop(); // stop video stream
    }
        ''')
    # make the provided HTML, part of the cell
    display(js) 
    # call the takePhoto() JavaScript function
    data = eval_js('takePhoto({})'.format(quality)) 

In [26]:
import IPython
import time
import sys
import numpy as np
import cv2
import base64
import logging

from google.colab import output
from PIL import Image
from io import BytesIO

from google.colab.patches import cv2_imshow

def data_uri_to_img(uri):
    """convert base64 image to numpy array"""
    try:
        image = base64.b64decode(uri.split(',')[1], validate=True)
        # make the binary image, a PIL image
        image = Image.open(BytesIO(image))
        # convert to numpy array
        image = np.array(image, dtype=np.uint8); 
        return image
    except Exception as e:
        logging.exception(e);print('\n')
        return None

def run_algo(imgB64):
    """
    in Colab, run_algo function gets invoked by the JavaScript, 
    that sends N images every second, one at a time.

    params:
        image: image
    """
    img = data_uri_to_img(imgB64)  
    if img is None:
        print("At run_algo(): image is None.")
        return

    try:
        # # Run detection
        # results = model.detect([img], verbose=1)
        # # Visualize results
        # r = results[0]    
        # visualize.display_instances(
        #     img, 
        #     r['rois'], 
        #     r['masks'], 
        #     r['class_ids'], 
        #     class_names, 
        #     r['scores']
        #     )
      while(True):
        faces = face_cascade.detectMultiScale(img, 1.3, 5)     
        for (x,y,w,h) in faces:
          if w > 130: 

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) #resize to 224x224
            
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            #img_pixels /= 255
            #employee dictionary is using preprocess_image and it normalizes in scale of [-1, +1]
            img_pixels /= 127.5
            img_pixels -= 1
            
            captured_representation = model.predict(img_pixels)[0,:]
            
            found = 0
            for i in employees:
              employee_name = i
              representation = employees[i]
              
              similarity = findCosineSimilarity(representation, captured_representation)
              if(similarity < 0.30):
                cv2.putText(img, employee_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                
                found = 1
                break
                
            #connect face and text
            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
          
            if(found == 0): #if found image is not in employee database
              cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        
        # cv2_imshow('img',img)
        cv2_imshow(img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
          break

    except Exception as e:
        logging.exception(e)
        print('\n')

In [ ]:
# register this function, so JS code could call this
output.register_callback('notebook.run_algo', run_algo)

# put the JS code in cell and run it
take_photo()